In [1]:
import folium

# Créer la carte
m = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

# Définir le HTML pour le marqueur avec un ID unique
marker_html = """
<div id="custom-marker" style="background-color: blue; width: 20px; height: 20px; border-radius: 50%; cursor: pointer;">
</div>
"""

# Ajouter le marqueur à la carte en utilisant un DivIcon avec le HTML défini
marker = folium.Marker([48.8566, 2.3522], icon=folium.DivIcon(html=marker_html)).add_to(
    m
)

# Injecter le script JavaScript pour changer la couleur du marqueur au clic
script = """
<script>
document.addEventListener("DOMContentLoaded", function() {
    var marker = document.getElementById("custom-marker");
    marker.addEventListener("click", function() {
        // Changer la couleur du marqueur
        if (marker.style.backgroundColor === "blue") {
            marker.style.backgroundColor = "red";
        } else {
            marker.style.backgroundColor = "blue";
        }
    });
});
</script>
"""

# Ajouter le script à la carte
m.get_root().html.add_child(folium.Element(script))

# Enregistrer la carte
m.save("map.html")

In [1]:
import pandas as pd

In [2]:
vanne = pd.read_excel("data/VANNE.xlsx")
cana = pd.read_excel("data/can.xlsx")



In [3]:
df_commune = cana[cana["COMMUNE"]=="Leforest"]

In [7]:
combined_nodes = pd.concat([df_commune["ID_NOEUD_1"], df_commune["ID_NOEUD_2"]])
vanne_in_cana = vanne[vanne["ID_NOEUD"].isin(combined_nodes)]
vanne_in_cana

,ID_NOEUD,ID_VANNE,DIAMETRE,ID_CANA_1,ID_CANA_2
1555,73766,152421,700.0,52303.0,52302.0
30510,119966,152420,400.0,52299.0,52301.0
30772,120150,152419,400.0,52297.0,52298.0
37256,33805,152422,400.0,52304.0,52300.0


In [5]:


combined_cana = pd.concat([vanne_in_cana["ID_CANA_1"], vanne_in_cana["ID_CANA_2"]])

cana_in = cana[cana["ID_CANA"].isin(combined_cana)]
cana_in

,Unnamed: 0,ID_CANA,ID_NOEUD_1,ID_NOEUD_2,LONGUEUR_EN_M,DIAMETRE,COMMUNE,MATERIAU,LATITUDE_1,LONGITUDE_1,LATITUDE_2,LONGITUDE_2
16169,16169,52303,22134,73766,2.040,700.0,Leforest,Acier,50.437210,3.073237,50.437193,3.073226
143651,143651,52300,33805,119965,3.047,400.0,Leforest,Fonte Ductile,50.437211,3.073229,50.437196,3.073200
144273,144273,52302,73766,58778,3.210,700.0,Leforest,Acier,50.437193,3.073226,50.437166,3.073208
144429,144429,52298,119965,120150,0.352,400.0,Leforest,Fonte Ductile,50.437196,3.073200,50.437198,3.073196
144718,144718,52299,119965,119966,3.596,400.0,Leforest,Fonte Ductile,50.437196,3.073200,50.437171,3.073193
144862,144862,52304,22134,33805,0.588,400.0,Leforest,Fonte Ductile,50.437210,3.073237,50.437211,3.073229
145119,145119,52297,120150,135647,3.936,400.0,Leforest,Fonte Ductile,50.437198,3.073196,50.437211,3.073144
145131,145131,52301,119966,58778,1.180,400.0,Leforest,Fonte Ductile,50.437171,3.073193,50.437166,3.073208


In [8]:
#
import matplotlib.pyplot as plt

# plt.style.use(['science', 'notebook'])
print(plt.style.available)

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']
